# Algoritmos de optimización - Seminario<br>

Nombre y Apellidos:

- Julio Emanuel Suriano Bryk
- José Fernández López

Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO

Problema:

> 1.  **Sesiones de doblaje**
> 2.  Organizar los horarios de partidos de La Liga
> 3.  Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

## Problema 1. Organizar sesiones de doblaje(I)

- Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.
- Los actores de doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.
- No es posible grabar más de 6 tomas por día.
- El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

(\*) La respuesta es obligatoria


(\*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.


In [1]:
# A completar!

Modelo para el espacio de soluciones<br>
(\*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


**Justificación:** Vamos a utilizar una matriz (lista de lista) para representar la tabla de tomas y actores, donde cada item dentro de la lista representa un toma y a su vez, cada toma contiene los actores que participan.

```python
example_schedule = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], # toma 1 con actores 1, 2, 3, 4 y 5
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], # toma 1 con actores 3, 4 y 5
    ...
]
```

Para representar el resultado final, es decir la planificacion de sesiones por dia, vamos a utilizar de nuevo una matriz.

```python
result = [
  [0,1,2], # dia 1 con las tomas 1, 2 y 3
  [3,5], # toma 2 con los actores 1 y 4
]
```


Según el modelo para el espacio de soluciones<br>
(\*)¿Cual es la función objetivo?

(\*)¿Es un problema de maximización o minimización?


**Justificación:** Es un problema de minimización donde la función objetivo es obtener una planificación de los actores por día que contemple el menor coste posible.


Funciones Base para los algoritmos


In [36]:
import random

# app constant
max_takes_per_day = 6
cost_per_actor = 10

# Taken from demo
test_schedule = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
]


def get_actors_in_day(planning: list, schedule: list[list]) -> list:
    """Get actors a given day

    Args:
        planning (list): array of takes in a day
        schedule (list[list]): array of actors per take, where each item represents one take

    Returns:
        list: actors in planning
    """
    # Obtain array of actors per take
    actors_per_take = [schedule[take] for take in planning]

    # Union of all actors in that day: ACTORS_TAKE_1 OR ACTORS_TAKE_2 OR ACTORS_TAKE_N
    return [int(any(i)) for i in zip(*actors_per_take)]


def get_cost(planning: list[list], schedule: list[list]) -> int:
    """Calculate total cost for a given planning

    Args:
        planning (list[list]): array of takes, where each item represents a day
        schedule (list[list]): array of actors per take, where each item represents one take

    Returns:
        int: total cost
    """

    cost = 0

    for day in planning:
        actors_in_day = get_actors_in_day(day, schedule)

        # Count the actors in that day and multiply by cost
        cost += actors_in_day.count(1) * cost_per_actor

    return cost


def get_planning(schedule: list[list]) -> list[list]:
    """Get a random planning given a schedule

    Args:
        schedule (list[list]): schedule of sessions with actors to record

    Returns:
        list[list]: possible random planning
    """
    # Get empty planning
    planning = []

    # Use schedule as a set of the ids
    schedule_set = set(range(len(schedule)))

    # continue until we schedule all the takes
    while len(schedule_set) > 0:
        # max amount of takes per day
        max_amount = (
            max_takes_per_day
            if max_takes_per_day <= len(schedule_set)
            else len(schedule_set)
        )

        # having different takes by day
        takes_amount = random.choice(range(1, max_amount)) if max_amount > 1 else 1

        # obtain take id from schedule
        takes = random.sample(list(schedule_set), takes_amount)

        # add them to the planning by day
        planning.append(takes)

        # remove takes from schedule
        schedule_set -= set(takes)

    return planning


test_planning = get_planning(test_schedule)
display(test_planning, get_cost(test_planning, test_schedule))

[[7, 22],
 [27],
 [10, 1],
 [0, 17, 8, 15],
 [6, 16, 26, 14],
 [29],
 [4],
 [19],
 [12, 3, 28, 11],
 [23, 13],
 [24],
 [21],
 [20],
 [2],
 [18, 5],
 [9],
 [25]]

710

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

Diseña un algoritmo para resolver el problema por fuerza bruta


Respuesta


Version Directa


In [124]:
import itertools
import math

schedule_demo = [
    [0, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 0],
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0],
    [1, 1, 1, 1],
    [1, 1, 1, 0],
]


def get_all_planning(takes):
    # if there is not takes remaining we return empty plannings
    if len(takes) == 0:
        return [[]]

    res = []

    # iterate over all possible combinations within the takes
    for i in range(1, min(len(takes), max_takes_per_day) + 1):
        for combination in itertools.combinations(takes, i):
            # get the rest of takes outside the combination
            remaining_takes = [take for take in takes if take not in combination]

            # calculate new planning for remaining takes
            for remaining_takes_planning in get_all_planning(remaining_takes):
                # create new planning based on original combination and planning
                new_planning = [list(combination)] + remaining_takes_planning

                # sort the planning to avoid duplication
                sorted_planning = sorted(new_planning, key=sorted)

                # if not existing add it to the possible results
                if sorted_planning not in res:
                    res.append(sorted_planning)
    return res


def get_planning_BF(schedule: list[list]) -> list[list]:
    best_cost = math.inf
    best_planning = []

    all_planning = get_all_planning(range(len(schedule)))

    for curr_plan in all_planning:
        curr_cost = get_cost(curr_plan, schedule)

        if curr_cost < best_cost:
            best_cost = curr_cost
            best_planning = curr_plan

    return best_planning


result = get_planning_BF(schedule_demo)
display("BF planning", result)
display("Cost", get_cost(result, schedule_demo))

'BF planning'

[[0], [1, 3, 4, 5, 6, 7], [2]]

'Cost'

70

In [100]:
def get_combination(n: int, r: int) -> list[int]:
    """Combination without repetition

    Args:
        n (int): amount of items
        r (int): elements in groups

    Returns:
        list[int]: combinations of items
    """
    div = math.factorial(r) * math.factorial(n - r)
    return math.factorial(n) / div


get_combination(3, 2) + get_combination(3, 1)

6.0

Calcula la complejidad del algoritmo por fuerza bruta


Respuesta


True

In [115]:
import random
import math


def probability(temp: int, dist: int) -> bool:
    return random.random() < math.exp(-1 * dist / temp)


def dec_temp(temp: int) -> int:
    return temp * 0.9999


def split_list(items: list, n_parts: int, max_amount: int) -> list:
    """Split items into different parts"""
    # copy of the list to avoid reference modification
    items_copy = items[:]

    # initialize parts with empty lists
    parts = [[] for _ in range(n_parts)]

    # continue until all the items have been properly distributed
    while items_copy:
        # pop item from array
        item = items_copy.pop()

        # use only available arrays
        available_parts = [
            index for index, value in enumerate(parts) if len(value) < max_amount
        ]

        # get random index and add item
        i = random.choice(available_parts)
        parts[i].append(item)

    return parts


def get_closest(planning: list[list]) -> list[list]:
    """Get closest random planning by swapping takes"""
    # Making a deep copy of the list to avoid object modification
    planning_copy = [lst[:] for lst in planning]

    # get random days from planning
    i, j = random.sample(range(len(planning)), 2)

    # group all takes into a single array
    all_takes = planning_copy[i] + planning_copy[j]

    # distribute them between the days
    planning_copy[i], planning_copy[j] = split_list(all_takes, 2, max_takes_per_day)

    return planning_copy


# Using Simulated Annealing heuristic --> https://en.wikipedia.org/wiki/Simulated_annealing
def get_planning_SA(schedule: list[list], temp=10000000) -> list[list]:
    curr_plan = get_planning(schedule)
    curr_plan_dist = get_distance(curr_plan, schedule)

    display("initial planning", curr_plan)
    display("Distance", get_distance(curr_plan, schedule))

    # max values for possible result
    best_plan = []
    best_plan_dist = 10e10000

    while temp > 0.0001:
        closest = get_closest(curr_plan)
        closest_dist = get_distance(closest, schedule)

        if closest_dist < best_plan_dist:
            best_plan = closest
            best_plan_dist = closest_dist

        dist_diff = abs(curr_plan_dist - closest_dist)
        if closest_dist < curr_plan_dist or probability(temp, dist_diff):
            curr_plan = closest
            curr_plan_dist = closest_dist

        temp = dec_temp(temp)

    return list(filter(lambda day: len(day) > 0, best_plan))


asd = [
    [0, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 0],
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0],
    [1, 1, 1, 1],
    [1, 1, 1, 0],
]

result = get_planning_SA(asd)
display("SA planning", result)
display("Distance", get_cost(result, asd))

'initial planning'

[[2, 1, 0, 5], [7], [4], [6], [3]]

'Distance'

180

'SA planning'

[[5, 2, 7], [4, 6, 3, 0, 1]]

'Distance'

70

(\*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta


Respuesta


In [ ]:
# A completar!

(\*)Calcula la complejidad del algoritmo


Respuesta


In [ ]:
# A completar!

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios


Respuesta


In [49]:
import random


def generate_schedule(n_days: int, n_actor: int) -> list[list]:
    # generate empty schedule
    schedule = [[] for _ in list(range(n_days))]

    # iterate over amount of days
    for i in range(n_days):
        # get random numbers of actors
        actors = random.sample(range(n_actor), random.randint(1, n_actor))

        # generate boolean array to store
        take_schedule = [int(j in actors) for j in range(n_actor)]

        # save in schedule
        schedule[i] = take_schedule

    return schedule


display(generate_schedule(10, 4))

[[0, 0, 1, 1],
 [1, 1, 0, 1],
 [1, 0, 0, 0],
 [1, 1, 1, 1],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1],
 [1, 1, 1, 0],
 [1, 1, 0, 1],
 [1, 0, 0, 1]]

Aplica el algoritmo al juego de datos generado


Respuesta


In [ ]:
# A completar!

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo


Respuesta


Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño


Respuesta
